In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from positionCorrection import positionCorrection as correct
from mpl_toolkits.mplot3d import Axes3D
import math

In [3]:
snapData = np.genfromtxt('data/data_spacing.txt')
snapshots = snapData[:,0]
scale = snapData[:,1]
redshift = snapData[:,2]


In [4]:
# reading in the data for all subhalos in identified groups
snaps = np.array([135, 131, 126, 117, 103, 93, 84, 75])
reds = np.array([redshift[snapshots == i][0] for i in snaps])
scales = 1/(1+reds)

primFiles = ['data/data_primaries_massAnalogs_'+str(i)+'.csv' for i in snaps]
secoFiles = ['data/data_secondaries_massAnalogs_'+str(i)+'.csv' for i in snaps]


In [5]:
#######################################
# Mass cuts for bound LMC/SMC analogs #
#######################################
numberBoundAnalogs = []

primaryCols = ['Group Number', 'Subhalo ID','Redshift', 'Mass at z=0', 'Max Mass',
           'Max Mass Snap', 'Redshift at Max Mass', 'Rvir at z=0', 'Rvir at Max Mass', 'Pos x',
           'Pos y', 'Pos z', 'Vel x', 'Vel y', 'Vel z', 'AM Stellar Mass']
secondaryCols = ['Group Number', 'Subhalo ID','Redshift', 'Mass at z=0', 'Max Mass',
           'Max Mass Snap', 'Redshift at Max Mass', 'Rvir at z=0', 'Rvir at Max Mass', 'Pos x',
           'Pos y', 'Pos z', 'Vel x', 'Vel y', 'Vel z', 'AM Stellar Mass','Escape Speed z=0','Escape Speed Max Mass']

for i in range(len(snaps)):
    #################################
    # velocity and separation cuts! #
    #################################
    prims = pd.read_csv(primFiles[i])
    secos = pd.read_csv(secoFiles[i])
    
    # finding scaled separation for each halo pair
    primaryPos = np.column_stack([prims['Pos x'],prims['Pos y'],prims['Pos z']])
    secondaryPos = np.column_stack([secos['Pos x'],secos['Pos y'],secos['Pos z']])
    correctedSeparations = np.array([np.linalg.norm(i) for i in np.array(correct(primaryPos,secondaryPos))])
    scaledSeparation = correctedSeparations/prims['Rvir at Max Mass'].values

    # finding scaled velocity for each halo pair
    primaryVel = np.column_stack([prims['Vel x'],prims['Vel y'],prims['Vel z']])
    secondaryVel = np.column_stack([secos['Vel x'],secos['Vel y'],secos['Vel z']])
    relativeVelocity = np.array([np.linalg.norm(i) for i in (primaryVel-secondaryVel)])
    scaledVelocity = relativeVelocity/secos['Escape Speed z=0'].values

    # creating masks to include only "bound" halos
    PosCut = (scaledSeparation <= 1)
    VelCut = (scaledVelocity <= 1)
    
    # applying bound cuts to 
    boundCut = PosCut&VelCut
    prims_boundCut = prims[boundCut]
    secos_boundCut = secos[boundCut]
    numberBoundAnalogs.append(len(prims_boundCut))
    
    LMCFileToSave = 'data/data_primaries_boundAnalogs_'+str(snaps[i])+'.csv'
    SMCFileToSave = 'data/data_secondaries_boundAnalogs_'+str(snaps[i])+'.csv'
    
    primsData = pd.DataFrame(data = prims_boundCut, columns=primaryCols)
    primsData.to_csv(LMCFileToSave,index=False,header=True)
    
    secosData = pd.DataFrame(data = secos_boundCut, columns=secondaryCols)
    secosData.to_csv(SMCFileToSave,index=False,header=True)
    

In [7]:
print(numberBoundAnalogs)
print(snaps)

[53, 44, 59, 44, 37, 24, 20, 7]
[135 131 126 117 103  93  84  75]
